In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)
#====================================================================
# calling data
trans = transforms.Compose([transforms.Resize((224,224)),
                            transforms.RandomHorizontalFlip(p=0.5), #data augmentation
                            transforms.RandomVerticalFlip(p=0.5),   #data augmentation
                            transforms.RandomAffine(15),            #data augmentation
                            transforms.ToTensor(), 
                            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = torchvision.datasets.ImageFolder(root='/home/gus/catvsdog',
                                            transform=trans)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=32,
                                          shuffle=True,
                                          drop_last=True,
                                          num_workers=2)
testset = torchvision.datasets.ImageFolder(root='/home/gus/cattest', 
                                           transform=trans)
testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=32,
                                         shuffle=True,
                                         drop_last=True,
                                         num_workers=2)
#====================================================================
# declaring hyperparameters
lr = 1e-3
#====================================================================
# constructiong AlexNet model
class VGGF(nn.Module):
    def __init__(self):
        super(VGGF,self).__init__()

        self.layer1 = nn.Sequential(
            # input shape = (?, 3, 224, 224)
            # Conv -> (?, 64, 54, 54) 54.25 -> 54
            # Pool -> (?, 64, 27, 27)
            torch.nn.Conv2d(3,64,11,stride=4,padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),
            torch.nn.LocalResponseNorm(64,alpha=1e-4,beta=0.75,k=2)      #LRN
        )
        torch.nn.init.normal_(self.layer1[0].weight,mean=0,std=0.01)    
        torch.nn.init.constant_(self.layer1[0].bias,0)
        self.layer2 = nn.Sequential(
            # input shape = (?, 64, 27, 27)
            # Conv -> (?, 256, 27, 27)
            # Pool -> (?, 256, 13, 13)
            torch.nn.Conv2d(64,256,5,stride=1,padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),
            torch.nn.LocalResponseNorm(256,alpha=1e-4,beta=0.75,k=2)     #LRN
        )
        torch.nn.init.normal_(self.layer2[0].weight,mean=0,std=0.01)    
        torch.nn.init.constant_(self.layer2[0].bias,1)
        self.layer3 = nn.Sequential(
            # input shape = (?, 256, 13, 13)
            # Conv -> (?, 256, 13, 13)
            torch.nn.Conv2d(256,256,3,stride=1,padding=1),
            torch.nn.ReLU())
        torch.nn.init.normal_(self.layer3[0].weight,mean=0,std=0.01)    
        torch.nn.init.constant_(self.layer3[0].bias,0)
        self.layer4 = nn.Sequential(
            # input shape = (?, 256, 13, 13)
            # Conv -> (?, 256, 13, 13)
            torch.nn.Conv2d(256,256,3,stride=1,padding=1),
            torch.nn.ReLU()
        )
        torch.nn.init.normal_(self.layer4[0].weight,mean=0,std=0.01)    
        torch.nn.init.constant_(self.layer4[0].bias,1)
        self.layer5 = nn.Sequential(
            # input shape = (?, 256, 13, 13)
            # Conv -> (?, 256, 13, 13)
            # Pool -> (?, 256, 6, 6)
            torch.nn.Conv2d(256,256,3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2)
        )
        torch.nn.init.normal_(self.layer5[0].weight,mean=0,std=0.01)    
        torch.nn.init.constant_(self.layer5[0].bias,1)
        self.layer6 = nn.Sequential(
            torch.nn.Dropout(p=0.5),                          #Implementing Dropout
            torch.nn.Linear(6*6*256,4096,bias=True),
            torch.nn.ReLU()
        )
        torch.nn.init.normal_(self.layer6[1].weight,mean=0,std=0.01)    
        torch.nn.init.constant_(self.layer6[1].bias,1)
        self.layer7 = nn.Sequential(
            torch.nn.Dropout(p=0.5),                          #Implementing Dropout
            torch.nn.Linear(4096,4096,bias=True),
            torch.nn.ReLU()
        )
        torch.nn.init.normal_(self.layer7[1].weight,mean=0,std=0.01)    
        torch.nn.init.constant_(self.layer7[1].bias,1)
        self.layer8 = nn.Sequential(
            torch.nn.Linear(4096,2,bias=True)
        )                         
        torch.nn.init.normal_(self.layer8[0].weight,mean=0,std=0.01)    
        torch.nn.init.constant_(self.layer8[0].bias,1)
        
    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=self.layer4(out)
        out=self.layer5(out)
        out=out.view(out.size(0),-1)
        out=self.layer6(out)
        out=self.layer7(out)
        out=self.layer8(out)
        return out
    
    
model = VGGF().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=0.9,weight_decay=5e-4)
# Using SGD with L2 weight decay 0.0005, momentum 0.9
for epoch in range(50):
    avg_cost = 0
    for X,Y in trainloader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/50
        break
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 0.0146232769
[Epoch:    2] cost = 0.0186377708
[Epoch:    3] cost = 0.038924098
[Epoch:    4] cost = 0.0168744456
[Epoch:    5] cost = 0.0881678089
[Epoch:    6] cost = 0.0652063191
[Epoch:    7] cost = 0.127668887
[Epoch:    8] cost = 0.0777117908
[Epoch:    9] cost = 0.0266165938
[Epoch:   10] cost = 0.0298444722
[Epoch:   11] cost = 0.0393521823
[Epoch:   12] cost = 0.0674315766
[Epoch:   13] cost = 0.025094917
[Epoch:   14] cost = 0.0472633578
[Epoch:   15] cost = 0.0283666551
[Epoch:   16] cost = 0.0182624739
[Epoch:   17] cost = 0.050219994
[Epoch:   18] cost = 0.0208130497
[Epoch:   19] cost = 0.0640331581
[Epoch:   20] cost = 0.0498352423
[Epoch:   21] cost = 0.0374015383
[Epoch:   22] cost = 0.0387890302
[Epoch:   23] cost = 0.0381313413
[Epoch:   24] cost = 0.0405852422
[Epoch:   25] cost = 0.041121304
[Epoch:   26] cost = 0.022743335
[Epoch:   27] cost = 0.0315007605
[Epoch:   28] cost = 0.0136848772
[Epoch:   29] cost = 0.0411438234
[Epoch:   30] cost =

In [ ]:
with torch.no_grad():
    a = 0
    for X,Y in testloader:
        X = X.to(device)
        Y = Y.to(device)
        prediction = model(X)
        correct_prediction = torch.argmax(prediction, 1) == Y
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())
        a += 1
        if a == 10:
            break

In [ ]:
torch.cuda.empty_cache()